---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [12]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
        State    Region Name
    0  Alabama      Auburn
    1  Alabama      Florence
    2  Alabama      Jacksonville
    
    etc.
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data = []
    
    with open('university_towns.txt') as f:
        content = f.readlines()
        
    content = [x.strip() for x in content]
   
    for line in content:
        if "[edit]" in line:
            position = line.index('[edit]')
            state = line[:position]
        else:
            try:
                position = line.index('(')
                region = line[:position].strip()
            except:
                region = line
                
            data.append([state,region])
        
    unt_df = pd.DataFrame(data, columns = ['State', 'RegionName'])
    
    return unt_df

In [39]:
def load_gdp_file():
    """ Load GDP file and returns a dataframe with records since 2000q1
    """
    
    
    # columns A:C contain the annual data
    # TO-DO: year is loaded as a float not as an int; look for dtype in the documentation
    # and https://trendct.org/2016/08/05/real-world-data-cleanup-with-python-and-pandas/
    
    # gdp_y = pd.read_excel('gdplev.xls',sheetname=0,skiprows=7, 
    #                         parse_cols="A:C", index_col=0,
    #                         names=['year','GDP','GDP 2009'])
    
    # columns E:G contain the quaterly seasonally adjusted annual data
    # GDP column = GDP in billions of current dollars
    # GDP 2009 = GDP in billions of chained 2009 dollars
    gdp_qdf = pd.read_excel('gdplev.xls',sheetname=0, 
                             skiprows=7, parse_cols="E:G", 
                             index_col=0, names=['GDP','GDP 2009'])
    
    # strip to data from first quarter of 2001 onwards
    gdp_qdf = gdp_qdf['2000q1':]
    gdp_qdf ['diff'] = gdp_qdf['GDP 2009'].diff()
    
    return gdp_qdf
    


def find_recession(pd_series):
    '''Check if there is a recession and returns the recession start and the recession end
       
       Definition: A recession start is defined as the beginning of a two quarter period 
       in which both quarters have GDP decline. The start of recession is the first quarter 
       of at least 2 quarters of GDP decline
       
       This function converts the series 'pd_series' in a boolean list, 
       where 0 is a decrease in GDP and 1 means a increase in GDP
       the recession will be a string equal to 00...11'''
    
    recession_found, start_year = False, -1
    recession_string = '0011'
    
    series_copy = pd_series.copy()
    pd_series = pd_series.apply(lambda x: 0 if x<0 else 1)
    series_list = pd_series.tolist()
    series_string = ''.join(map(str, series_list))
    
    position = series_string.index(recession_string)
    
    # recession starts the following year to the last GDP increase
    recession_position = series_string[:position].rindex('1') + 1
    recession_start = pd_series.index[recession_position]
    
    # recession end is recession positon + len(recession_string) + two consecutives quarters.
    # the lenght of the reccesion_string, that is 4, hence we have to add 6
    recession_end = pd_series.index[recession_position + 5]
    
        
    return recession_start, recession_end


def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp_qdf = load_gdp_file()
    
    # use function find_recession to find the year
    recession_start, recession_end = find_recession(gdp_qdf['diff'])
    return recession_start

get_recession_start()

'2008q3'

In [40]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp_qdf = load_gdp_file()
    recession_start, recession_end = find_recession(gdp_qdf['diff'])
    return recession_end

get_recession_end()

'2009q4'

In [7]:
def find_recession_bottom(pd_series):
    
    '''Check if there is a recession and returns the recession start and the recession end
       
       Definition: A recession start is defined as the beginning of a two quarter period 
       in which both quarters have GDP decline. The start of recession is the first quarter 
       of at least 2 quarters of GDP decline
       
       This function converts the series 'pd_series' in a boolean list, 
       where 0 is a decrease in GDP and 1 means a increase in GDP
       the recession will be a string equal to 00...11'''
    
    recession_found, start_year = False, -1
    recession_string = '0011'
    
    pd_series = pd_series.apply(lambda x: 0 if x<0 else 1)
    series_list = pd_series.tolist()
    series_string = ''.join(map(str, series_list))
    
    position = series_string.index(recession_string)
    recession_bottom = pd_series.index[position + 1]
        
    return recession_bottom


def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp_qdf = load_gdp_file()
    recession_bottom = find_recession_bottom(gdp_qdf['diff'])
    
    return recession_bottom

get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    
    Need help?
    https://stackoverflow.com/questions/41661451/pandas-compute-the-half-yearly-average-of-monthly-collected-house-sales
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/CSgStATMEee9mg5CyCleJA
    
    '''
    
    
    df = pd.read_csv('City_Zhvi_AllHomes.csv', index_col=0, skiprows=0)
    
    #slice columns with loc
    time_cols_df = df.loc[:,'2000-01':'2016-08']
    
    # change columns names type to datetime
    time_cols_df = time_cols_df.rename(columns=pd.to_datetime)
    
    # perform a resample ('Q' = quarter) and calculate the mean
    mean_df = time_cols_df.resample('Q', axis=1).mean()
    
    # change the column names back to string
    mean_df = mean_df.rename(columns = lambda x: '{:}q{:}'.format(x.year, x.quarter))
    
    # add States and Regions to the Dataframe and replace States codes with its names
    mean_df['RegionName'] = df['RegionName']
    mean_df['State'] = df['State'].replace(states)
    
    
    # set the index to ['State','RegionName']
    mean_df = mean_df.set_index(['State','RegionName'])
    
    return mean_df

In [32]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivalent to a
    reduced market loss).
    
    help?
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/iCSsLhrvEee0thK6z3XwTA
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/pY_v4ryDEeaN_w7XVB3P7A
    https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/1Fkg-ryCEeaIRw7T1E5tHA
    
    '''
    
    hdf =convert_housing_data_to_quarters()
    rec_start=get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()
    
    # Price ratio = Quarter before Recession Start / Recession Bottom Quarter
    # ojo esto está mal! Es el período anterior a rec_start
    # 2008q3 --> 2008q2!!!
    qtr_bfr_rec_start = str(pd.Period(rec_start,'Q')-1).lower()
    hdf['PriceRatio'] = hdf[qtr_bfr_rec_start].div(hdf[rec_bottom])
    
    # to test the hypothesis (University towns have their mean housing prices
    # split hdf in two sets: university towns data and the rest
    # since ul has a list of university town, we can use it
    # converting ul to a list of tuples [(State1, RegionName1), etc]
    ul_list = [(state, region) for state,region in ul.itertuples(index=False)]
    university_towns = hdf.loc[ul_list].dropna()
    non_university_towns = hdf.loc[~hdf.index.isin(ul_list)].dropna()

    
    # run ttest on the price ration column to test the hypothesis is null (invalid)
    # the result of ttest pvalue determines if the null hypothesis is true or not
    # 
    # "null hypothesis" is a general statement or default position
    # that there is a relationship between two measured phenomena
    #
    different, pvalue = ttest_ind(university_towns['PriceRatio'], non_university_towns['PriceRatio'])
    
    # if pvalue<0.01, we reject the null hypothesis 
    # and hence it's possible that there is a relation ship between 
    # {housing prices being affected by recession} and {being located in a university town or not}
    
    # if pvalue>0.01 we cannot reject the null hypothesis
    # and hence there is no relationship between them
    # 
    different = pvalue < 0.01
    utm = university_towns['PriceRatio'].mean()
    nutm = non_university_towns['PriceRatio'].mean()
    better = 'university town' if utm < nutm else 'non-university-town'
    
    return (different, pvalue, better)


run_ttest()

(True, 0.0051648663279195324, 'university town')